In [1]:
import pickle
import cv2
import os
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
      

def getFiles(NAME):
    cascade_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    DATABASE_PATH = "../../Datastore/Face/"+NAME+"/hasFaces"
    images=[]
    labels=[]
    global MAX_IMAGES 
    local_MAX_IMAGES=MAX_IMAGES
    # Loop through all files in the directory
    for filename in os.listdir(DATABASE_PATH):
        local_MAX_IMAGES-=1      
        if filename.endswith('.jpg'):
            # Construct the file path
            filepath = os.path.join(DATABASE_PATH, filename)
            # Read the image
            img = cv2.imread(filepath)
            
            ##Filtering for better recognition
            greyscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = cascade_classifier.detectMultiScale(greyscale, 1.3, 5)            
            for (x,y,w,h) in faces:
                roi = img[y:y+ROI_SIZE_XY, x:x+ROI_SIZE_XY]                        
            images.append(roi)
            labels.append(NAMES.index(NAME))
    
        if (local_MAX_IMAGES<1):
            break
    return (images,labels)
def train_svm(features, labels):
    # Split data into training and testing subsets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

    # Train SVM classifier
    svm_classifier = svm.SVC()
    svm_classifier.fit(X_train, y_train)

    return svm_classifier, X_test, y_test
def extract_hog_features(images):
    hog = cv2.HOGDescriptor()
    features = []
    
    good=[]
    for img in images:
        hog_features = hog.compute(img)
#        features.sort(key=lambda x: x.distance)
        features.append(hog_features.flatten())
        
    for feature in features[:1000]:
        good.append(feature)
    for i in good:
        print(len(i))
    return (good)




######START PROGRAM

ROI_SIZE_XY=200
MAX_IMAGES = 50

NAMES = ["Harry","Phil"]#,"Maureen"]
numerical_labels=[]
for name in NAMES:
    try:
        index = NAMES.index(name)
        numerical_labels.append(index)
    except ValueError:
        print("Element not found")

###Gather images and labels in arrays
images = []
labels = []
for name in NAMES:
    img, lab = getFiles(name)
    images+=(img)
    labels+=(lab)
    print(len(img),"Images added for",name)
print(len(images),"Images added in total ")

##hog features

# Step 5: Testing/Evaluation
def test_svm(svm_classifier, X_test, y_test):
    
    predictions = svm_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy:", accuracy)
    
    
# Extract HoG features
print("DONE\n\nExtracting HoG features")
hog_features = extract_hog_features(images)

print("DONE\n\nTraining SVM classifier")
# Train SVM classifier
svm_classifier, X_test, y_test= train_svm(hog_features, labels)

print("DONE\n\nTesting SVM Classifier")
# Test SVM classifier
print(y_test)
test_svm(svm_classifier, X_test, y_test)


print("DONE\n\nSaving Classifier")
# Create the label mapping dictionary
label_mapping = {numerical_labels: label for numerical_labels, label in zip(numerical_labels, NAMES)}
# Save the SVM classifier
with open('svm_classifier.pkl', 'wb') as f:
    pickle.dump(svm_classifier, f)

# Save the label encoding mapping
with open('label_mapping.pkl', 'wb') as f:
    pickle.dump(label_mapping, f)
    
print("Saved\n\nCOMPLETED")



FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../Datastore/Face/Harry/hasFaces'